# Linear Regression 1

Train simple well formatted "libsvm" data using Spark MLlib Linear Regression Model

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
# Load training data
# libsvm format, not csv
training = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

In [5]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

This is the format that Spark expects. Two columns with the names "label" and "features". 

The "label" column then needs to have the numerical label, either a regression numerical value, or a numerical value that matches to a classification grouping. 

The feature column has inside of it a vector of all the features that belong to that row. 

In [6]:
# These are the default values for the featuresCol, labelCol, predictionCol
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

In [7]:
# Fit the model to the training data
lrModel = lr.fit(training)

In [9]:
# Print the coefficients and intercept for linear regression
print("The Linear Regression Coefficients are: \n {}".format(str(lrModel.coefficients))) # For each feature...
print('\n')
print("The Intercept Line is: \n {}".format(str(lrModel.intercept)))

The Linear Regression Coefficients are: 
 [0.0073350710225801715,0.8313757584337543,-0.8095307954684084,2.441191686884721,0.5191713795290003,1.1534591903547016,-0.2989124112808717,-0.5128514186201779,-0.619712827067017,0.6956151804322931]


The Intercept Line is: 
 0.14228558260358093


### Summarize the model over the training set and print out some metrics

In [10]:
trainingSummary = lrModel.summary

In [11]:
trainingSummary.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-11.011130022096554|
| 0.9236590911176538|
|-4.5957401897776675|
|  -20.4201774575836|
|-10.339160314788181|
|-5.9552091439610555|
|-10.726906349283922|
|  2.122807193191233|
|  4.077122222293811|
|-17.316168071241652|
| -4.593044343959059|
|  6.380476690746936|
| 11.320566035059846|
|-20.721971774534094|
| -2.736692773777401|
| -16.66886934252847|
|  8.242186378876315|
|-1.3723486332690233|
|-0.7060332131264666|
|-1.1591135969994064|
+-------------------+
only showing top 20 rows



In [13]:
print("MSE Score: {}".format(trainingSummary.meanSquaredError))
print()
print("RMSE Score: {}".format(trainingSummary.rootMeanSquaredError))
print()
print("R2 Score: {}".format(trainingSummary.r2))
print()
print("MAE Score: {}".format(trainingSummary.meanAbsoluteError))
print()
print("Explained Variance Score: {}".format(trainingSummary.explainedVariance))

MSE Score: 103.28843028724194

RMSE Score: 10.16309157133015

R2 Score: 0.027839179518600154

MAE Score: 8.145215527783876

Explained Variance Score: 2.957808103742484


## Train/Test Splits

In [14]:
all_data = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

In [15]:
# training/test - 70/30 splits. 
train_data, test_data = all_data.randomSplit([0.7, 0.3])

In [17]:
train_data.show(5)

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-26.805483428483072|(10,[0,1,2,3,4,5,...|
|-26.736207182601724|(10,[0,1,2,3,4,5,...|
| -23.51088409032297|(10,[0,1,2,3,4,5,...|
|-23.487440120936512|(10,[0,1,2,3,4,5,...|
|-22.949825936196074|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 5 rows



In [18]:
test_data.show(5)

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.571478869743427|(10,[0,1,2,3,4,5,...|
|-28.046018037776633|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-19.872991038068406|(10,[0,1,2,3,4,5,...|
|-18.845922472898582|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 5 rows



In [21]:
# Train data has 365 rows
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                365|
|   mean| 0.5176647505395394|
| stddev| 10.287203801040997|
|    min|-26.805483428483072|
|    max|  27.78383192005107|
+-------+-------------------+



In [22]:
# Test data has 136 rows
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                136|
|   mean|-0.4429877502003075|
| stddev| 10.405503834742106|
|    min|-28.571478869743427|
|    max| 27.111027963108548|
+-------+-------------------+



In [25]:
train_label = train_data.select('label')

In [26]:
train_label.show(5)

+-------------------+
|              label|
+-------------------+
|-26.805483428483072|
|-26.736207182601724|
| -23.51088409032297|
|-23.487440120936512|
|-22.949825936196074|
+-------------------+
only showing top 5 rows



In [33]:
train_features = train_data.select('features')

In [34]:
train_features.show(5)

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 5 rows



# Now we only train on the train_data

In [29]:
correct_model = lr.fit(train_data)

Now we can directly get a .summary object using the evaluate method:

In [30]:
test_results = correct_model.evaluate(test_data)

In [31]:
test_results.residuals.show()


+-------------------+
|          residuals|
+-------------------+
|-28.308589538406128|
|  -26.2965555701744|
|-21.671408682693656|
|-22.163020139922033|
| -22.05880669116706|
| -19.65274828801603|
|-15.459684780392053|
| -18.34058167957884|
|-17.859309232179505|
|-18.983529805283393|
|-15.784335745514808|
|-18.311241497023538|
|-17.621554697273034|
|-13.919745811051923|
|-15.660835619810507|
| -13.91548243618662|
|-14.193919276812185|
|-14.073904596898673|
|-11.461342057218824|
|-10.597836219266707|
+-------------------+
only showing top 20 rows



In [32]:
print("MSE Score: {}".format(test_results.meanSquaredError))
print()
print("RMSE Score: {}".format(test_results.rootMeanSquaredError))
print()
print("R2 Score: {}".format(test_results.r2))
print()
print("MAE Score: {}".format(test_results.meanAbsoluteError))
print()
print("Explained Variance Score: {}".format(test_results.explainedVariance))

MSE Score: 108.215991765365

RMSE Score: 10.402691563502447

R2 Score: -0.006862941694760094

MAE Score: 8.33981324700433

Explained Variance Score: 4.311857692382774


### test this model against unlabeled data, in this case, transform().

In [35]:
unlabeled_data = test_data.select('features')

In [36]:
unlabeled_data.show(5)

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 5 rows



In [37]:
predictions = correct_model.transform(unlabeled_data)

In [38]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|-0.26288933133729675|
|(10,[0,1,2,3,4,5,...| -1.7494624676022335|
|(10,[0,1,2,3,4,5,...|  1.6139260669044442|
|(10,[0,1,2,3,4,5,...|  2.2900291018536256|
|(10,[0,1,2,3,4,5,...|  3.2128842182684787|
|(10,[0,1,2,3,4,5,...|  1.3775347220114016|
|(10,[0,1,2,3,4,5,...|  -2.034515576491291|
|(10,[0,1,2,3,4,5,...|   1.275182053702823|
|(10,[0,1,2,3,4,5,...|  1.1671022108684017|
|(10,[0,1,2,3,4,5,...|  2.7220995298306603|
|(10,[0,1,2,3,4,5,...| -0.1671768202797651|
|(10,[0,1,2,3,4,5,...|   2.449232169452978|
|(10,[0,1,2,3,4,5,...|  1.8408696646497313|
|(10,[0,1,2,3,4,5,...| -1.5176389823792942|
|(10,[0,1,2,3,4,5,...|  0.8386827100593187|
|(10,[0,1,2,3,4,5,...| -0.8472758167445062|
|(10,[0,1,2,3,4,5,...|    1.15399121270757|
|(10,[0,1,2,3,4,5,...|  1.5153288080424845|
|(10,[0,1,2,3,4,5,...| -1.0063143238140362|
|(10,[0,1,2,3,4,5,...| -1.532516